In [1]:
# -*- coding: utf-8 -*-
import tensorflow as tf
import numpy as np
import datetime
from tensorflow.contrib.learn.python.learn.datasets.mnist import DataSet
from tensorflow.contrib.learn.python.learn.datasets import base
from tensorflow.python.framework import dtypes
from tensorflow.examples.tutorials.mnist import input_data
from OWMLayer_2Layers import OWMLayer
import os
import pdb
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # ignore warning
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # use gpu

# Parameters
# ==================================================
tf.flags.DEFINE_integer("num_checkpoints", 3, "Number of checkpoints to store (default: 5)")
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
tf.app.flags.DEFINE_string("buckets", "", "")
tf.app.flags.DEFINE_string("checkpointDir", "", "oss info")
tf.flags.DEFINE_integer("num_class", 10, "")
tf.flags.DEFINE_integer("batch_size", 40, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("epoch", 20, "")
FLAGS = tf.flags.FLAGS
# ==================================================

mnist = input_data.read_data_sets("./data/MNIST_data/", one_hot=True)


/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
import pdb
def next_batch(l, n,m): 
    final = [l[i * n:(i + 1) * n,:] for i in range((len(l) + n - 1) // n )]
    #Repeating the frazme segmentation for different epochs
    if m >= len(final): 
        m = m % len(final)
        if m == 0:
            np.random.shuffle(l)
            final = [l[i * n:(i + 1) * n,:] for i in range((len(l) + n - 1) // n )]
    return(final[m]) 

#Heuristic metric: Weights Pattern after training each task
from math import*
#1. Euclidean distance
def euclidean_distance(x,y):
    return sqrt(sum(pow(a-b,2) for a, b in zip(x, y)))

#2. Cosine Similarity
def square_rooted(x):
    return round(sqrt(sum([a*a for a in x])),3)
def cosine_similarity(x,y):
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return round(numerator/float(denominator),3)

#3. Jaccard similarity
def jaccard_similarity(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

#4. Using SequenceMatcher    
from difflib import SequenceMatcher

In [3]:
import pdb, pprint
from numpy import count_nonzero
def split_mnist(mnist, cond):
    sets = ["train", "validation", "test"]
    sets_list = []
    for set_name in sets:
        this_set = getattr(mnist, set_name)
        maxlabels = np.argmax(this_set.labels, 1)
        sets_list.append(DataSet(this_set.images[cond(maxlabels),:], this_set.labels[cond(maxlabels)],
                                 dtype=dtypes.uint8, reshape=False))
    return base.Datasets(train=sets_list[0], validation=sets_list[1], test=sets_list[2])


def train(mnist_list):
    # Training
    # ==================================================
    g1 = tf.Graph()
    middle = 50
    with g1.as_default():
        OWM = OWMLayer([[784 + 1, middle], [middle + 1, 10]], seed_num=79)

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    test_array = []
    Gradients = []
    Weights = []
    Tasks_Accuracy = []
    loss_after_task = []
    with tf.Session(graph=g1, config=config) as sess1:
        # Initialize all variables
        init = [tf.global_variables_initializer(), tf.local_variables_initializer()]
        sess1.run(init)
        task_num = len(mnist_list)
        for j in range(0, task_num):
            print("Training Disjoint MNIST %d" % (j + 1))

            # Update the parameters
            epoch_owm = FLAGS.epoch
            batch_size_owm = FLAGS.batch_size
            print('Task performs classification between {0}'.format(set(np.argmax(mnist_list[j].train.labels,axis=1))))
            all_data = len(mnist_list[j].train.labels[:])
            all_step = all_data*epoch_owm//batch_size_owm
            for current_step in range(all_step):
                lamda = current_step/all_step
                current_step = current_step+1
                batch_xs, batch_ys = mnist_list[j].train.next_batch(batch_size_owm)
                feed_dict = {
                    OWM.input_x: batch_xs,
                    OWM.input_y: batch_ys,
                    OWM.lr_array: np.array([[0.2]]),
                    OWM.alpha_array: np.array([[0.9 * 0.001 ** lamda, 0.6]]),
                }
                acc, loss, grads_var, _, = sess1.run([OWM.accuracy, OWM.loss, OWM.grads , OWM.back_forward], feed_dict,)
                '''
                if current_step % (all_step // 4) == 0:
                    print("Train->>>Task: [{:d}/{:d}] Step: {:d}/{:d} Train: loss: {:.2f}, acc: {:.2f}  %"
                          .format(j+1, task_num,current_step*epoch_owm // all_step+1, epoch_owm, loss, acc * 100))
                '''
            print('Shape of P1 = {}. Rank of P1 = {}. Shape of P2 = {}. Rank of P2={}'.format(sess1.run(OWM.P1, feed_dict,).shape, np.linalg.matrix_rank(sess1.run(OWM.P1, feed_dict,)), sess1.run(OWM.P2, feed_dict,).shape, np.linalg.matrix_rank(sess1.run(OWM.P2, feed_dict,))))
            print('Number of non-zero in P1 = {}, in P2={}'.format(count_nonzero(sess1.run(OWM.P1, feed_dict,)), count_nonzero(sess1.run(OWM.P2, feed_dict,))))
            
            loss_after_task.append(loss)
            Gradients.append([grads_var[0][0], grads_var[0][1]])
            Weights.append([grads_var[1][0], grads_var[1][1]])  
            print("Test on Previous Datasets:")
            correct = []
            Per_task_acc_holder = {}
            for i_test in range(task_num):
                feed_dict = {
                    OWM.input_x: mnist_list[i_test].test.images[:],
                    OWM.input_y: mnist_list[i_test].test.labels[:],
                }
                accu, = sess1.run([OWM.accuracy], feed_dict)
                Per_task_acc_holder['Task' + str(i_test+1)] = accu
                correct.append(accu)
            pprint.pprint(Per_task_acc_holder)
            Tasks_Accuracy.append(correct)
            test_accu = np.mean(correct)
            test_array.append(test_accu)
    return(Gradients,Weights, Tasks_Accuracy, loss_after_task)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython import display
def main(_):
    # Create 2 disjoint MNIST datasets
    dataset01 = split_mnist(mnist, lambda x: x < 2)
    dataset23 = split_mnist(mnist, lambda x: (1 < x) & (x < 4))
    dataset45 = split_mnist(mnist, lambda x: (3 < x) & (x < 6))
    dataset67 = split_mnist(mnist, lambda x: (5 < x) & (x < 8))
    dataset89 = split_mnist(mnist, lambda x: (7 < x) & (x < 10))
    mnist_list = [dataset01, dataset23, dataset45, dataset67, dataset89]
    for dtask in mnist_list:
        print('Rank of the training task {} is {}'.format(i,np.linalg.matrix_rank(dtask)))
    Gradients, Weights, Tasks_Accuracy,loss_after_task = train(mnist_list)
    print('Tasks_Accuracy = {}, loss_after_task = {}'.format(Tasks_Accuracy, loss_after_task))
    
    import pdb
    import seaborn as sns
    from matplotlib.pyplot import figure
    figure(figsize=(4, 4.5))
    axs = [plt.subplot(1,1,1)]#, None, None]
    fontsize = 13
    average_acc=[]
    for i in range(len(Tasks_Accuracy)):
        average_acc.append(np.mean(Tasks_Accuracy[i]))
    
    print(Tasks_Accuracy)
    colorVal = ['r','g','b','c','m','k','y','tomato','m','c']
    marker=["v","^","<",">","*","s","h","x",".","+"]
    n_tasks = 5
    mean_vals = np.array(Tasks_Accuracy)
    for j in range(n_tasks):
        label = "$\mathcal{T}$=%g"%(j+1)
        axs[0].plot(range(1,n_tasks+1), mean_vals[:, j], marker=marker[j], c=colorVal[j], label=label)
    axs[0].plot(range(1,n_tasks+1), average_acc, marker=marker[j+1], c=colorVal[j+1], label='Average')
    axs[0].legend(loc='upper right',prop={'size': fontsize}, bbox_to_anchor=(1.05, -0.25), ncol=3)
    axs[0].set_xlabel("Trained tasks($\mathcal{T}$).",fontsize=fontsize)
    axs[0].set_ylabel('Accuracy(x100%).',fontsize=fontsize)
    axs[0].set_xticks(range(1,n_tasks+1))
    axs[0].set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1.0])
    axs[0].tick_params(labelsize='large', width=3)
    axs[0].set_title('OWM for AWID dataset', fontsize=fontsize)
    #plt.legend(bbox_to_anchor=(0.78, 0.66),bbox_transform=plt.gcf().transFigure)
    plt.gcf().tight_layout()
    sns.despine()
    plt.savefig('AWID_OWM.pdf',bbox_inches = "tight")
    
    
    def flatten(x):
        try:
            it = iter(x)
        except TypeError:
            yield x
        else:
            for i in it:
                for j in flatten(i):
                    yield j
    Flatten_weights=[]
    for i in range(len(Gradients)):
        Flatten_weights.append(list(flatten(Gradients[i]))) 
    
    #Creating the Hessian matrix from the gradients
    import math
    Hessian_matrix = []
    Hessian_row = []
    Rank_Hessian_matrix = []
    for k in range(len(Flatten_weights)):
        for i in range(len(Flatten_weights[k])):
            partial_deriv_1 = (math.ceil(Flatten_weights[k][i]*1e10)/1e10)
            for j in range(len(Flatten_weights[k])):
                partial_deriv_2 = (math.ceil(Flatten_weights[k][j]*1e10)/1e10)
                Hessian_row.append(np.around(partial_deriv_1*partial_deriv_2))
            Hessian_matrix.append(Hessian_row)
            Hessian_row = []
        rank_Hessian_matrix = np.linalg.matrix_rank(np.asarray(Hessian_matrix))
        Rank_Hessian_matrix.append(rank_Hessian_matrix)
        print('Rank of the Hessian Matrix after task=',k,'is :',rank_Hessian_matrix)  
    
    #Rank_Hessian_matrix = [22, 53, 101, 157, 268]  #Calculated by uncommenting above code and then coped here
    
    figure(figsize=(3, 5))
    axs = [plt.subplot(2,1,1)]#, None, None]
    for i in range(1, 2):
        axs.append(plt.subplot(2, 1, i+1, sharex=axs[0]))
    fontsize = 13

    colorVal = ['r','g','b','c','m','k','y']
    marker=["v","^","<",">","*","s"]
    #loss_after_task[-1]=5
    plots = [loss_after_task,Rank_Hessian_matrix]
    ylabel = ['Task loss ($L_{\mu+1}$)', 'Hessian rank R(H)']
    ytick = [[0,1.0,2.0,3.0,4.0,5.0, 6.0], [0,300,600,900, 1200]]
    title = ['Loss', 'Hessian rank']
    for i in range(2):
        axs[i].plot(range(1,n_tasks+1), plots[i], marker=marker[i], c=colorVal[i])
        axs[i].set_ylabel(ylabel[i],fontsize=fontsize)
        axs[i].set_yticks(ytick[i])
        axs[i].tick_params(labelsize='large', width=3)
        axs[i].grid()
    axs[-1].set_xticks(range(1,n_tasks+1))
    axs[-1].set_xlabel("Trained tasks($\mathcal{T}$).",fontsize=fontsize)
    plt.tight_layout()
    sns.despine()
    plt.savefig('AWID_OWM_evalcongestion.pdf',bbox_inches = "tight")
    
    
    model_weights_save=[]
    for i in range(len(Weights)):
        model_weights_save.append(list(flatten(Weights[i])))
        
    print("--------------Euclidean distance--------------------")
    Eu_distance = []
    for i in range(1,n_tasks):
        eu_dist = euclidean_distance(list(flatten(model_weights_save[i-1])),list(flatten(model_weights_save[i])))
        Eu_distance.append(eu_dist)
        print("Between task {0} and {1} : {2}".format(i-1,i,eu_dist))

    print("--------------Cosine distance--------------------")
    Cos_distance = []
    for i in range(1,n_tasks):
        cos_dist = cosine_similarity(list(flatten(model_weights_save[i-1])),list(flatten(model_weights_save[i])))
        Cos_distance.append(cos_dist)
        print("Between task {0} and {1} : {2}".format(i-1,i,cos_dist))


    print("--------------Jaccard similarity--------------------")
    Jac_distance = []
    for i in range(1,n_tasks):
        jac_dist = jaccard_similarity(list(flatten(model_weights_save[i-1])),list(flatten(model_weights_save[i])))
        Jac_distance.append(jac_dist)
        print("Between task {0} and {1} : {2}".format(i-1,i,jac_dist))

    print("--------------Sequence Matcher--------------------")
    Seq_matcher = []
    for i in range(1,n_tasks):
        seq_matcher = SequenceMatcher(None,list(flatten(model_weights_save[i-1])),list(flatten(model_weights_save[i]))).ratio()
        Seq_matcher.append(seq_matcher)
        print("Between task {0} and {1} : {2}".format(i-1,i,seq_matcher))
    
    figure(figsize=(3, 8))
    plt.subplots_adjust(hspace=0.25,
                        wspace=0.35)
    axs = [plt.subplot(4,1,1)]#, None, None]
    for i in range(1, 4):
        axs.append(plt.subplot(4, 1, i+1, sharex=axs[0]))
    fontsize = 13
    colorVal = ['r','g','b','c','m','k','y']
    marker=["v","^","<",">","*","s"]
    plots = [Eu_distance,Cos_distance,Jac_distance,Seq_matcher]
    ylabel = ['Distance value', 'Ratio value', 'Ratio value','Ratio value']
    ytick = [[0.0, 3.0, 6.0, 9.0, 12.0], [0, 0.5, 1.0],[0, 0.4, 0.8, 1.2], [0, 0.4, 0.8, 1.2]]
    title = ['Euclidean Distance', 'Cosine Similarity', 'Jaccard Similarity', 'SequenceMatcher']
    for i in range(4):
        axs[i].plot(range(2,n_tasks+1), plots[i], marker=marker[i], c=colorVal[i])
        axs[i].set_ylabel(ylabel[i],fontsize=fontsize)
        axs[i].set_yticks(ytick[i])
        axs[i].tick_params(labelsize='large', width=3)
        axs[i].grid()
        axs[i].set_title(title[i])
    axs[-1].set_xticks(range(2,n_tasks+1))
    axs[-1].set_xlabel("Trained tasks($\mathcal{T}$).",fontsize=fontsize)
    plt.tight_layout()
    sns.despine()
    plt.savefig('AWID_OWM_Similaritymetric.pdf',bbox_inches = "tight")
    
    #Heuristic metric: Weights Pattern between task 0 and last task 
    n_tasks = len(model_weights_save)
    eu_dist = euclidean_distance(list(flatten(model_weights_save[0])),list(flatten(model_weights_save[-1])))
    print("Euclidean distance between task {0} and {1} : {2}".format(0,len(model_weights_save)-1,eu_dist))

    cos_dist = cosine_similarity(list(flatten(model_weights_save[0])),list(flatten(model_weights_save[-1])))
    print("Cosine angle between task {0} and {1} : {2}".format(0,len(model_weights_save)-1,cos_dist))

    jac_dist = jaccard_similarity(list(flatten(model_weights_save[0])),list(flatten(model_weights_save[-1])))
    print("Jaccard similarity between task {0} and {1} : {2}".format(0,len(model_weights_save)-1,jac_dist))

    seq_matcher = SequenceMatcher(None,list(flatten(model_weights_save[0])),list(flatten(model_weights_save[-1]))).ratio()
    print("Sequence angle between task {0} and {1} : {2}".format(0,len(model_weights_save)-1,seq_matcher)) 

if __name__ == '__main__':
    tf.app.run()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Training Disjoint MNIST 1
Task performs classification between {0, 1}
Shape of P1 = (785, 785). Rank of P1 = 769. Shape of P2 = (51, 51). Rank of P2=50
Number of non-zero in P1 = 373495, in P2=2601
Test on Previous Datasets:
{'Task1': 0.9990544, 'Task2': 0.0, 'Task3': 0.0, 'Task4': 0.0, 'Task5': 0.0}
Training Disjoint MNIST 2
Task performs classification between {2, 3}
Shape of P1 = (785, 785). Rank of P1 = 749. Shape of P2 = (51, 51). Rank of P2=50
Number of non-zero in P1 = 447677, in P2=2601
Test on Previous Datasets:
{'Task1': 0.99101657,
 'Task2': 0.94074434,
 'Task3': 0.0,
 'Task4': 0.0,
 'Task5': 0.0}
Training Disjoint MNIST 3
Task performs classification between {4, 5}
Shape of P1 = (785, 785). Rank of P1 = 734. Shape o